In [ ]:
## ABRIR DATASET ACTUALIZADO
import pickle

with open("dataset_actualizado.pkl", "rb") as f:
    datos = pickle.load(f)

print(datos)
print(datos.keys())
print(datos["data"])
print(datos["distancias"])
print(datos["tiempos"])
print(datos["costes"])


{'data': [{'nodo': 'parking', 'nombre': 'ES08191 Rubi', 'direccion': 'Ctra. Molins de Rei, 251, Km. 8, 08191 Rubí, Barcelona', 'coordenadas': (41.46930679260032, 2.029808746545829), 'w': 0}, {'nodo': 'carga', 'nombre': 'ES08840 Viladecans', 'direccion': 'Av. del Segle XXI, 63, 08840 Viladecans, Barcelona', 'coordenadas': (41.3207455, 2.0347543), 'w': 0}, {'nodo': 'carga', 'nombre': 'ES08272 Sant Fruitós de Bages', 'direccion': 'Carrer Sakura, 16B, NAVE 16A, 08272 Sant Fruitós de Bages, Barcelona', 'coordenadas': (41.77803590470732, 1.867141253552206), 'w': 0}, {'nodo': 'carga', 'nombre': 'ES08650 Sallent', 'direccion': "Avinguda Can Montcau, 1-3-5, 08186 Lliçà d'Amunt, Barcelona", 'coordenadas': (41.60289038509304, 2.2569771240231096), 'w': 0}, {'nodo': 'carga', 'nombre': 'ES08186 Lliça de Amunt', 'direccion': "Carrer d'Anselm Clavé, 62 AB, 08186 Lliçà d'Amunt, Barcelona", 'coordenadas': (41.6230442, 2.2365817), 'w': 0}, {'nodo': 'descarga', 'nombre': 'ES08040 Barcelona', 'direccion': 

In [3]:
## CREAR EL CONJUNTO DE NODOS ##
base_operativa_ficticia=-1
N = [i for i,d in enumerate(datos["data"]) if d['nodo']=="carga"]
F = [i for i,d in enumerate(datos["data"]) if d['nodo']=="cargador"]
V = list(range(len(datos["data"])))+[base_operativa_ficticia]
base_operativa = [i for i,d in enumerate(datos["data"]) if d['nodo']=="parking"][0]
descarga = [i for i,d in enumerate(datos["data"]) if d['nodo']=="descarga"][0]

print(V)
print(N)
print(F)
print(base_operativa)
print(descarga)
print(base_operativa_ficticia)


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, -1]
[1, 2, 3, 4]
[6, 7, 8, 9, 10]
0
5
-1


In [40]:
# M-GRANDE 
M = 1000

In [ ]:
# CREACIÓN DE ARRAYS
import numpy as np
c_0 = np.array(datos["costes"])
w_0 = np.array([d["w"] for d in datos["data"]])
tiempos_0 = np.array(datos["tiempos"])

[[ 0.         14.14285714 31.60285714 20.63142857 20.80285714 12.46285714
   5.07428571  8.52       11.12571429 15.09428571 10.99714286  0.        ]
 [14.14285714  0.         45.15428571 31.21714286 32.20285714 23.69142857
   9.76285714 19.54285714 19.38857143  7.26857143 20.55428571 14.14285714]
 [31.60285714 45.15428571  0.         32.43428571 30.20571429 30.69428571
  35.4        31.73142857 35.37428571 46.58571429 26.10857143 31.60285714]
 [20.63142857 31.21714286 32.43428571  0.          2.40857143  8.17714286
  25.20857143 12.31714286 11.88       27.23142857 28.32857143 20.63142857]
 [20.80285714 32.20285714 30.20571429  2.40857143  0.          8.61428571
  25.58571429 12.85714286 13.11428571 28.65428571 27.68571429 20.80285714]
 [12.46285714 23.69142857 30.69428571  8.17714286  8.61428571  0.
  17.05714286  4.24285714  5.49428571 20.73428571 20.85428571 12.46285714]
 [ 5.07428571  9.76285714 35.4        25.20857143 25.58571429 17.05714286
   0.         12.89142857 14.58857143 12

In [ ]:
# AÑADIR EL INDICE -1 (BASE OPERATIVA FICTICIA)
#c
fila_nueva_c = c_0[0, :].reshape(1, -1)
c = np.vstack((c_0, fila_nueva_c))
columna_nueva_c = c[:, 0].reshape(-1, 1)
c = np.hstack((c, columna_nueva_c))
#w
w=np.append(w_0,0)
#tiempos
fila_nueva_t = tiempos_0[0, :].reshape(1, -1)
tiempos = np.vstack((tiempos_0, fila_nueva_t))
columna_nueva_t = tiempos[:, 0].reshape(-1, 1)
tiempos = np.hstack((tiempos, columna_nueva_t))


In [45]:
## PROGRAMA PRINCIPAL
import gurobipy as gp
from gurobipy import GRB

# Creamos el modelo
m = gp.Model("problema1")

# Creamos las variables
x = m.addVars(V,V, vtype=GRB.BINARY, name="x")
z = m.addVars(F, vtype=GRB.BINARY, name="z")
t = m.addVars(V, lb=0.0, ub= 15*60, vtype=GRB.CONTINUOUS, name="t")
m.update()

# Definimos la función objetivo
obj = gp.quicksum(c[i, j] * x[i, j] for i in V for j in V if i != j) + gp.quicksum(w[i] * z[i] for i in F)
m.setObjective(obj, GRB.MINIMIZE)

# Definimos las restricciones
## Ponemos que en la base operativa el tiempo es = 0
m.addConstr(t[base_operativa] == 0, name="r0")
# r1
for i in V:
    if i not in F and i!= base_operativa_ficticia:
        m.addConstr(gp.quicksum(x[i, j] for j in V if j != i) == 1, name=f"r1_{i}")
#r2
m.addConstr(gp.quicksum(x[base_operativa,j] for j in V if j not in [base_operativa, base_operativa_ficticia])==1, name="r2")
#r3
m.addConstr(gp.quicksum(x[i,base_operativa_ficticia] for i in V if i not in [base_operativa, base_operativa_ficticia])==1, name="r3")
#r4
m.addConstr(gp.quicksum(x[descarga, i] for i in N)==0, name="r4")
#r5
for j in V:
    if j not in [base_operativa_ficticia, base_operativa]:
        m.addConstr(gp.quicksum(x[i,j] for i in V if i!=j)-gp.quicksum(x[j,h] for h in V if h!=j)==0, name=f"r5_{j}")
#r6
for i in V:
    for j in V:
        if i!=j and j!=base_operativa:
            m.addConstr(t[i]+tiempos[i,j]-t[j]<=M*(1-x[i,j]), name=f"r6_{i}_{j}")     
#r7
m.addConstr(gp.quicksum(x[i,j] for i in (N+[base_operativa]) for j in F if i!=j)==1, name="r7")
#r8
for i in F:
    for j in N:
        if i!=j:
            m.addConstr(x[i,j]<= z[i], name=f"r8_{i}_{j}")        
#r9
for i in N:
    for j in F:
        if i!=j:
            m.addConstr(x[i,j]<= z[j], name=f"r9_{i}_{j}") 
# r10
#Bloquea salidas del nodo ficticio de retorno (-1)
for j in V:
    m.addConstr(x[base_operativa_ficticia, j] == 0, name=f"r10_{j}")

m.update()
m.optimize()
a= m.ObjVal
print("Funcion objetivo", a)
for v in m.getVars():
    print(str(v.VarName)+"="+str(round(v.x, 2)))


Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[x86] - Darwin 21.6.0 21H1123)

CPU model: Intel(R) Core(TM) i5-5350U CPU @ 1.80GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 194 rows, 161 columns and 791 nonzeros
Model fingerprint: 0x94cebdaf
Variable types: 12 continuous, 149 integer (149 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+00, 2e+02]
  Bounds range     [1e+00, 9e+02]
  RHS range        [1e+00, 1e+03]
Presolve removed 45 rows and 31 columns
Presolve time: 0.02s
Presolved: 149 rows, 130 columns, 1171 nonzeros
Variable types: 10 continuous, 120 integer (120 binary)

Root relaxation: objective 5.373193e+01, 62 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   53.73193    0    8          -   53.73193      -     -    